![Erudio logo](../img/erudio-logo-small.png)

# Saving and Loading NumPy Arrays

While not part of Python itself, the NumPy array library forms the basis for nearly all numeric computation within Python.  A few core features of the Python language have been specialized to accomodate the NumPy community and library.  The most notable examples of the language definition being modified for the sake of NumPy are the extended slice notation and the matrix multiply operator.

The *stride* argument to slices was added in Python 1.4 (long ago), but was not used by Python lists, tuples, or strings until 2.3 (still a long time).  The use of commas within compound slice descriptions is not used anywhere in Python's standard library but exists so that NumPy (and later other libraries) can utilize it.  Similarly, the operator `@` is not used anywhere in Python itself or its standard library, but was added so that NumPy can use it to denote matrix multiplication (some other libraries have utilized it for other purposes since then).

In [2]:
import pickle
import numpy as np

In [4]:
%%bash
rm tmp/*

# Serializing with Pickle

Most Python objects, even those in extension libraries, can be serialized and deserialized with `pickle` module.  Classes are able to define a few protocol methods that allow them to interoperate with pickling.  For most purposes, pickle is fine for representing NumPy arrays.  Let us create one and roundtrip it.

In [5]:
arr = np.random.randint(1, 100, 1_000_000).reshape(100, 100, 100)
arr.shape

(100, 100, 100)

In [6]:
# Syntax extras for NumPy (matrix multiply dimensional slices)
arr[2:6, 4, 3:4] @ arr[8:9, 3, 4:8]

array([[1680,  308, 1120, 1204],
       [4380,  803, 2920, 3139],
       [3000,  550, 2000, 2150],
       [1680,  308, 1120, 1204]])

Dumping and loading a pickle of a NumPy array is the same as for any Python object.

In [7]:
pickle.dump(arr, open('tmp/arr.pkl', 'wb'))
arr2 = pickle.load(open('tmp/arr.pkl', 'rb'))
arr2[:2]

array([[[24, 37, 59, ..., 89,  8,  7],
        [67, 30, 24, ...,  4, 47, 21],
        [32, 22, 29, ..., 41, 35, 98],
        ...,
        [12, 88, 38, ..., 42, 60, 69],
        [97, 70, 16, ..., 72, 50, 39],
        [88, 89, 98, ..., 33, 45, 49]],

       [[11, 47, 24, ..., 82, 28,  6],
        [91, 12,  4, ..., 15, 90, 89],
        [34, 67, 84, ..., 76, 35, 59],
        ...,
        [69, 72, 92, ..., 90, 62, 56],
        [ 6, 67, 55, ..., 82, 46, 86],
        [87, 33, 35, ..., 32,  3, 50]]])

An advantage of pickles is that you might embed an array inside other structures, and pickle will handle that.

In [8]:
data = {'array': arr,
        'description': "A million random integers",
        'list': [5.4, 9.1, 3.4],
        'another_array': np.arange(5.0)}

data_bytes = pickle.dumps(data)
print(data_bytes[:35])

new_data = pickle.loads(data_bytes)
print("Description:", new_data['description'])
print("Extra data:", new_data['another_array'])

b'\x80\x04\x95\x90\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x05array\x94\x8c\x15numpy.core.'
Description: A million random integers
Extra data: [0. 1. 2. 3. 4.]


# Serializing with `np.savetxt()`

For NumPy arrays that are 1-D or 2-D, you can save them as delimited files with the `savetxt()` function.  This is a convenient way to save data to CSV or TSV that might be read by DataFrame libraries or similar tools.  However, multi-dimensional arrays need to be reduced to 2-D to be stored in this manner.

In [9]:
np.savetxt('tmp/arr-txt.tsv', 
           arr.reshape(100_000, 10),
           delimiter='\t',
           fmt='%d',
           header='Original shape: (100, 100, 100)',
           comments='# ')

In [11]:
%%bash
head tmp/arr-txt.tsv

# Original shape: (100, 100, 100)
24	37	59	87	45	14	2	80	90	96
35	68	62	24	66	22	38	59	96	7
58	13	61	38	89	93	70	76	1	25
98	98	99	70	72	50	67	3	83	14
34	53	65	48	84	54	72	39	92	16
31	76	34	77	55	74	48	6	28	15
99	77	41	74	36	84	55	94	34	92
21	28	14	4	82	72	20	61	7	13
48	49	30	53	96	10	39	25	94	58


# Serializing with `np.save()`

The native NumPy serialization format is very simple and directly represents arrays on disk.  An `.npy` file is *slightly* faster to write than a pickle, and is *slightly* smaller on disk.  These differences are minimal and are swamped by disk caching effects and data size, respectively.  The actual advantage of `.npy` is precisely what it *does not* do; reading a serialized array will never instantiate custom classes, will never execute arbitrary code, and will never contain structures other than arrays.

In [12]:
%time np.save('tmp/arr.npy', arr)

CPU times: total: 0 ns
Wall time: 3.51 ms


In [13]:
arr3 = np.load('tmp/arr.npy')
arr3[:2]

array([[[24, 37, 59, ..., 89,  8,  7],
        [67, 30, 24, ...,  4, 47, 21],
        [32, 22, 29, ..., 41, 35, 98],
        ...,
        [12, 88, 38, ..., 42, 60, 69],
        [97, 70, 16, ..., 72, 50, 39],
        [88, 89, 98, ..., 33, 45, 49]],

       [[11, 47, 24, ..., 82, 28,  6],
        [91, 12,  4, ..., 15, 90, 89],
        [34, 67, 84, ..., 76, 35, 59],
        ...,
        [69, 72, 92, ..., 90, 62, 56],
        [ 6, 67, 55, ..., 82, 46, 86],
        [87, 33, 35, ..., 32,  3, 50]]])

# Serializing with `np.savez` and `np.savez_compressed`

An enhancement to the `.npy` format is the `.npz` format.  This uses a zipfile wrapper to aggregate multiple arrays in the same file.  Again, pickle could do this by putting them inside a dict or a list; the restriction is exactly the advantage for some cases.  In general, the compressed version is to be preferred in almost all cases; for the last decade, the extra CPU cycles to perform compression have been almost always faster than the extra time required to write more data to disk.

In [14]:
np.savez('tmp/arr', arr, data['another_array'])
np.savez_compressed('tmp/arr-compress', arr, data['another_array'])

The multiple arrays are available in a dict-like interface, and simply named as `arr_0`, `arr_1`, and so on.  You must store any mapping to the variable names used for these by separate means.

In [15]:
arr_data = np.load('tmp/arr.npz')
for name in arr_data:
    print(name, arr_data[name].shape, arr_data[name].dtype)

print(arr_data['arr_1'])

arr_0 (100, 100, 100) int32
arr_1 (5,) float64
[0. 1. 2. 3. 4.]


# File sizes

We serialized the same data using several different formats.  The CPU times taken for all of them are neglibigle; there are some notable patterns in disk usage.  

In [17]:
%%bash
ls -la tmp

total 15912
drwxrwxrwx 1 laishaw laishaw     512 Feb 13 07:39 .
drwxrwxrwx 1 laishaw laishaw     512 Feb 13 07:38 ..
-rwxrwxrwx 1 laishaw laishaw 1277898 Feb 13 07:39 arr-compress.npz
-rwxrwxrwx 1 laishaw laishaw 3009490 Feb 13 07:39 arr-txt.tsv
-rwxrwxrwx 1 laishaw laishaw 4000128 Feb 13 07:39 arr.npy
-rwxrwxrwx 1 laishaw laishaw 4000546 Feb 13 07:39 arr.npz
-rwxrwxrwx 1 laishaw laishaw 4000163 Feb 13 07:39 arr.pkl


Notice the initially surprising fact that the text format is not the largest.  Because all of our integers were only two digits, they were each stored with two bytes for the numbers plus one for the delimiter.  In contrast, an int64 value requires 8 bytes to store uncompressed.  If the data contained values closer to `sys.maxsize`, i.e. 9,223,372,036,854,775,807, the size of the text version could easily become larger.

-------------
Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors